In [2]:
# from the official keras documentation: https://keras.io/examples/mnist_cnn/ get the MNIST part
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from keract import get_activations
import math
import time


batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#model definition
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


'''
layer_name = 'my_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(data)
'''

"""
A weighted version of keras.objectives.categorical_crossentropy

Variables:
    weights: numpy array of shape (C,) where C is the number of classes

Usage:
    weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
    loss = weighted_categorical_crossentropy(weights)
    model.compile(loss=loss,optimizer='adam')
"""
def weighted_categorical_crossentropy(weights, input_tensor):
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        
        return loss
    
    return loss

print('layer names:')
for layer in model.layers:
    print(layer.name)
    
get_first_layer_output = K.function([model.layers[0].input],
                                  [model.layers[0].output])
get_second_layer_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])

get_last_layer_output = K.function([model.layers[0].input],
                                  [model.layers[7].output])
print(len(model.layers))

weights = np.array([1,1,1,1,1,1,1,1,1,1])
input_tensor = Input(shape=(batch_size,))

model.compile(loss=weighted_categorical_crossentropy(weights,input_tensor),
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

epochs = 3
n_batches = math.floor( x_train.shape[0] / batch_size)

for e in range(epochs):
    
    for i in range(0,n_batches):
        imgs = x_train[i*batch_size:(i+1)*batch_size]
        labels = y_train[i*batch_size:(i+1)*batch_size]
        
        intermediates_before = get_first_layer_output([imgs])
        
        start_time1 = time.time()
        loss = model.train_on_batch(imgs,labels)
        timer1 = time.time() - start_time1
        print("epoch: " + str(e) + " batch " + str(i) + " loss: " + str(loss[0]) + " acc: " + str( 100*loss[1]))
        
        intermediates_after = get_first_layer_output([imgs])
        
        start_time2 = time.time()
        tmp = get_last_layer_output([imgs])[0]
        
        timer2 = time.time() - start_time2
        print('training time ' + str(timer1) + ' all layer time: ' + str(timer2) + ' shape: ' + str(tmp.shape))
    
    print('before')
    print(intermediates_before[0][0][0][0])
    print('after')
    print(intermediates_after[0][0][0][0])

'''
for i in range(batches):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train[idx]
    labels = y_train[idx] 
    loss = model.train_on_batch(imgs,labels)
    print(" loss: " + str(loss[0]) + " acc: " + str( 100*loss[1]))
'''   
    
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
layer names:
conv2d_3
conv2d_4
max_pooling2d_2
dropout_3
flatten_2
dense_3
dropout_4
dense_4
8
epoch: 0 batch 0 loss: 2.3175635 acc: 7.8125
training time 1.1076486110687256 all layer time: 0.05936932563781738 shape: (128, 10)
epoch: 0 batch 1 loss: 2.2415533 acc: 22.65625
training time 0.0225222110748291 all layer time: 0.005574941635131836 shape: (128, 10)
epoch: 0 batch 2 loss: 2.1374862 acc: 28.90625
training time 0.022604942321777344 all layer time: 0.0058023929595947266 shape: (128, 10)
epoch: 0 batch 3 loss: 2.0582056 acc: 35.9375
training time 0.022075653076171875 all layer time: 0.005417823791503906 shape: (128, 10)
epoch: 0 batch 4 loss: 1.9877467 acc: 35.15625
training time 0.02189016342163086 all layer time: 0.005617618560791016 shape: (128, 10)
epoch: 0 batch 5 loss: 1.8938826 acc: 39.84375
training time 0.022000789642333984 all layer time: 0.00596165657043457 shape: (128, 10)
epoch: 0 batch 6 loss: 1.

epoch: 0 batch 62 loss: 0.48905122 acc: 84.375
training time 0.017287015914916992 all layer time: 0.004300594329833984 shape: (128, 10)
epoch: 0 batch 63 loss: 0.3325061 acc: 90.625
training time 0.018769264221191406 all layer time: 0.004352569580078125 shape: (128, 10)
epoch: 0 batch 64 loss: 0.48752308 acc: 86.71875
training time 0.01722097396850586 all layer time: 0.0043256282806396484 shape: (128, 10)
epoch: 0 batch 65 loss: 0.3791962 acc: 92.1875
training time 0.017220020294189453 all layer time: 0.004446983337402344 shape: (128, 10)
epoch: 0 batch 66 loss: 0.42252165 acc: 89.84375
training time 0.016841888427734375 all layer time: 0.004199504852294922 shape: (128, 10)
epoch: 0 batch 67 loss: 0.6574448 acc: 77.34375
training time 0.016928911209106445 all layer time: 0.0043642520904541016 shape: (128, 10)
epoch: 0 batch 68 loss: 0.6191781 acc: 82.8125
training time 0.01737689971923828 all layer time: 0.004367828369140625 shape: (128, 10)
epoch: 0 batch 69 loss: 0.493394 acc: 84.375

epoch: 0 batch 125 loss: 0.24473669 acc: 94.53125
training time 0.017943620681762695 all layer time: 0.004197835922241211 shape: (128, 10)
epoch: 0 batch 126 loss: 0.12191232 acc: 96.09375
training time 0.01726078987121582 all layer time: 0.004031181335449219 shape: (128, 10)
epoch: 0 batch 127 loss: 0.2148695 acc: 94.53125
training time 0.016941070556640625 all layer time: 0.004265546798706055 shape: (128, 10)
epoch: 0 batch 128 loss: 0.16346046 acc: 96.875
training time 0.017228364944458008 all layer time: 0.004332542419433594 shape: (128, 10)
epoch: 0 batch 129 loss: 0.21067408 acc: 92.96875
training time 0.01812267303466797 all layer time: 0.0044252872467041016 shape: (128, 10)
epoch: 0 batch 130 loss: 0.2718116 acc: 89.84375
training time 0.017023801803588867 all layer time: 0.004177570343017578 shape: (128, 10)
epoch: 0 batch 131 loss: 0.23609957 acc: 93.75
training time 0.016955137252807617 all layer time: 0.004306793212890625 shape: (128, 10)
epoch: 0 batch 132 loss: 0.28586596

epoch: 0 batch 188 loss: 0.123314895 acc: 96.875
training time 0.01717829704284668 all layer time: 0.0044209957122802734 shape: (128, 10)
epoch: 0 batch 189 loss: 0.15101744 acc: 94.53125
training time 0.01681351661682129 all layer time: 0.004233360290527344 shape: (128, 10)
epoch: 0 batch 190 loss: 0.11005266 acc: 95.3125
training time 0.017132282257080078 all layer time: 0.004393577575683594 shape: (128, 10)
epoch: 0 batch 191 loss: 0.17555098 acc: 92.96875
training time 0.01703047752380371 all layer time: 0.004194736480712891 shape: (128, 10)
epoch: 0 batch 192 loss: 0.13202779 acc: 93.75
training time 0.017070531845092773 all layer time: 0.004325151443481445 shape: (128, 10)
epoch: 0 batch 193 loss: 0.29315534 acc: 92.1875
training time 0.016951322555541992 all layer time: 0.004182100296020508 shape: (128, 10)
epoch: 0 batch 194 loss: 0.2582217 acc: 90.625
training time 0.017529726028442383 all layer time: 0.004147768020629883 shape: (128, 10)
epoch: 0 batch 195 loss: 0.106562 acc:

epoch: 0 batch 251 loss: 0.108872026 acc: 98.4375
training time 0.01708221435546875 all layer time: 0.004308938980102539 shape: (128, 10)
epoch: 0 batch 252 loss: 0.22440246 acc: 93.75
training time 0.016959428787231445 all layer time: 0.004243135452270508 shape: (128, 10)
epoch: 0 batch 253 loss: 0.21212912 acc: 92.1875
training time 0.016895294189453125 all layer time: 0.004206180572509766 shape: (128, 10)
epoch: 0 batch 254 loss: 0.09133582 acc: 96.09375
training time 0.016765594482421875 all layer time: 0.004285097122192383 shape: (128, 10)
epoch: 0 batch 255 loss: 0.15171187 acc: 94.53125
training time 0.01710796356201172 all layer time: 0.004395961761474609 shape: (128, 10)
epoch: 0 batch 256 loss: 0.28416127 acc: 93.75
training time 0.017151832580566406 all layer time: 0.004220724105834961 shape: (128, 10)
epoch: 0 batch 257 loss: 0.06252152 acc: 98.4375
training time 0.01677989959716797 all layer time: 0.004183292388916016 shape: (128, 10)
epoch: 0 batch 258 loss: 0.15118511 ac

...

epoch: 2 batch 189 loss: 0.03801238 acc: 99.21875
training time 0.016826868057250977 all layer time: 0.004421234130859375 shape: (128, 10)
epoch: 2 batch 190 loss: 0.0093824165 acc: 100.0
training time 0.01692795753479004 all layer time: 0.0041522979736328125 shape: (128, 10)
epoch: 2 batch 191 loss: 0.022258764 acc: 99.21875
training time 0.016994476318359375 all layer time: 0.004213571548461914 shape: (128, 10)
epoch: 2 batch 192 loss: 0.10689309 acc: 96.875
training time 0.017610788345336914 all layer time: 0.00426793098449707 shape: (128, 10)
epoch: 2 batch 193 loss: 0.15532361 acc: 98.4375
training time 0.01751852035522461 all layer time: 0.004325389862060547 shape: (128, 10)
epoch: 2 batch 194 loss: 0.11810379 acc: 95.3125
training time 0.017383098602294922 all layer time: 0.004540920257568359 shape: (128, 10)
epoch: 2 batch 195 loss: 0.00805436 acc: 100.0
training time 0.017097949981689453 all layer time: 0.004512310028076172 shape: (128, 10)
epoch: 2 batch 196 loss: 0.058236495

epoch: 2 batch 252 loss: 0.10581018 acc: 97.65625
training time 0.017478227615356445 all layer time: 0.00417780876159668 shape: (128, 10)
epoch: 2 batch 253 loss: 0.09607599 acc: 96.09375
training time 0.01779007911682129 all layer time: 0.004343509674072266 shape: (128, 10)
epoch: 2 batch 254 loss: 0.044765405 acc: 98.4375
training time 0.01797652244567871 all layer time: 0.004245281219482422 shape: (128, 10)
epoch: 2 batch 255 loss: 0.05552397 acc: 97.65625
training time 0.01717376708984375 all layer time: 0.004171133041381836 shape: (128, 10)
epoch: 2 batch 256 loss: 0.09104519 acc: 97.65625
training time 0.01691269874572754 all layer time: 0.0042040348052978516 shape: (128, 10)
epoch: 2 batch 257 loss: 0.012909916 acc: 99.21875
training time 0.016737937927246094 all layer time: 0.004270792007446289 shape: (128, 10)
epoch: 2 batch 258 loss: 0.08007512 acc: 96.875
training time 0.018117904663085938 all layer time: 0.0043642520904541016 shape: (128, 10)
epoch: 2 batch 259 loss: 0.0347

epoch: 2 batch 314 loss: 0.052285243 acc: 99.21875
training time 0.01718592643737793 all layer time: 0.004207611083984375 shape: (128, 10)
epoch: 2 batch 315 loss: 0.043770473 acc: 97.65625
training time 0.017696142196655273 all layer time: 0.004363536834716797 shape: (128, 10)
epoch: 2 batch 316 loss: 0.020078924 acc: 99.21875
training time 0.016465187072753906 all layer time: 0.004293918609619141 shape: (128, 10)
epoch: 2 batch 317 loss: 0.061245065 acc: 96.875
training time 0.016985654830932617 all layer time: 0.004151821136474609 shape: (128, 10)
epoch: 2 batch 318 loss: 0.06892291 acc: 98.4375
training time 0.01688098907470703 all layer time: 0.0043833255767822266 shape: (128, 10)
epoch: 2 batch 319 loss: 0.02654656 acc: 99.21875
training time 0.01660299301147461 all layer time: 0.004236936569213867 shape: (128, 10)
epoch: 2 batch 320 loss: 0.07601623 acc: 96.875
training time 0.016569137573242188 all layer time: 0.004225969314575195 shape: (128, 10)
epoch: 2 batch 321 loss: 0.038

epoch: 2 batch 377 loss: 0.09843659 acc: 97.65625
training time 0.017427682876586914 all layer time: 0.004190206527709961 shape: (128, 10)
epoch: 2 batch 378 loss: 0.011305928 acc: 100.0
training time 0.017493009567260742 all layer time: 0.004255771636962891 shape: (128, 10)
epoch: 2 batch 379 loss: 0.023391988 acc: 100.0
training time 0.017163991928100586 all layer time: 0.004239797592163086 shape: (128, 10)
epoch: 2 batch 380 loss: 0.02211948 acc: 99.21875
training time 0.017258167266845703 all layer time: 0.0041005611419677734 shape: (128, 10)
epoch: 2 batch 381 loss: 0.02173864 acc: 99.21875
training time 0.01695561408996582 all layer time: 0.004187822341918945 shape: (128, 10)
epoch: 2 batch 382 loss: 0.15233217 acc: 95.3125
training time 0.016813278198242188 all layer time: 0.0042264461517333984 shape: (128, 10)
epoch: 2 batch 383 loss: 0.08423487 acc: 96.09375
training time 0.01695108413696289 all layer time: 0.004204750061035156 shape: (128, 10)
epoch: 2 batch 384 loss: 0.09845

epoch: 2 batch 440 loss: 0.04191071 acc: 99.21875
training time 0.017086267471313477 all layer time: 0.0042226314544677734 shape: (128, 10)
epoch: 2 batch 441 loss: 0.07643046 acc: 97.65625
training time 0.017267942428588867 all layer time: 0.004179477691650391 shape: (128, 10)
epoch: 2 batch 442 loss: 0.053510208 acc: 98.4375
training time 0.01687455177307129 all layer time: 0.004344940185546875 shape: (128, 10)
epoch: 2 batch 443 loss: 0.077983454 acc: 96.875
training time 0.01704096794128418 all layer time: 0.0042018890380859375 shape: (128, 10)
epoch: 2 batch 444 loss: 0.06729982 acc: 97.65625
training time 0.01761937141418457 all layer time: 0.0042688846588134766 shape: (128, 10)
epoch: 2 batch 445 loss: 0.046110433 acc: 98.4375
training time 0.016768217086791992 all layer time: 0.004172325134277344 shape: (128, 10)
epoch: 2 batch 446 loss: 0.008566316 acc: 100.0
training time 0.017474889755249023 all layer time: 0.004242420196533203 shape: (128, 10)
epoch: 2 batch 447 loss: 0.085

In [4]:
# from https://skymind.ai/wiki/generative-adversarial-network-gan
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("./gan/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

/home/test/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.578585, acc.: 46.88%] [G loss: 0.699700]


/home/test/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.354314, acc.: 78.12%] [G loss: 0.723980]
2 [D loss: 0.317373, acc.: 93.75%] [G loss: 0.717782]
3 [D loss: 0.287278, acc.: 100.00%] [G loss: 0.822010]
4 [D loss: 0.295507, acc.: 87.50%] [G loss: 0.922056]
5 [D loss: 0.282318, acc.: 96.88%] [G loss: 1.122179]
6 [D loss: 0.237372, acc.: 93.75%] [G loss: 1.292230]
7 [D loss: 0.177808, acc.: 100.00%] [G loss: 1.474605]
8 [D loss: 0.163413, acc.: 100.00%] [G loss: 1.670371]
9 [D loss: 0.147224, acc.: 100.00%] [G loss: 1.777976]
10 [D loss: 0.169061, acc.: 96.88%] [G loss: 2.014380]
11 [D loss: 0.113736, acc.: 100.00%] [G loss: 2.182868]
12 [D loss: 0.093973, acc.: 100.00%] [G loss: 2.331012]
13 [D loss: 0.065767, acc.: 100.00%] [G loss: 2.482884]
14 [D loss: 0.079054, acc.: 100.00%] [G loss: 2.598572]
15 [D loss: 0.062686, acc.: 100.00%] [G loss: 2.669461]
16 [D loss: 0.072377, acc.: 100.00%] [G loss: 2.699196]
17 [D loss: 0.045911, acc.: 100.00%] [G loss: 2.788925]
18 [D loss: 0.044632, acc.: 100.00%] [G loss: 2.855821]
19 [D l

...

8219 [D loss: 0.742295, acc.: 46.88%] [G loss: 0.915655]
8220 [D loss: 0.599937, acc.: 71.88%] [G loss: 0.959437]
8221 [D loss: 0.646179, acc.: 68.75%] [G loss: 0.843354]
8222 [D loss: 0.639359, acc.: 68.75%] [G loss: 0.865722]
8223 [D loss: 0.687919, acc.: 59.38%] [G loss: 0.927986]
8224 [D loss: 0.682441, acc.: 56.25%] [G loss: 0.891714]
8225 [D loss: 0.651070, acc.: 65.62%] [G loss: 0.900691]
8226 [D loss: 0.619518, acc.: 65.62%] [G loss: 0.955507]
8227 [D loss: 0.633264, acc.: 68.75%] [G loss: 0.903630]
8228 [D loss: 0.680787, acc.: 56.25%] [G loss: 0.824104]
8229 [D loss: 0.636691, acc.: 62.50%] [G loss: 0.864491]
8230 [D loss: 0.714663, acc.: 53.12%] [G loss: 0.983392]
8231 [D loss: 0.593216, acc.: 78.12%] [G loss: 0.896465]
8232 [D loss: 0.648973, acc.: 59.38%] [G loss: 0.911401]
8233 [D loss: 0.593404, acc.: 65.62%] [G loss: 0.908332]
8234 [D loss: 0.657486, acc.: 62.50%] [G loss: 0.817966]
8235 [D loss: 0.679627, acc.: 53.12%] [G loss: 0.852621]
8236 [D loss: 0.682457, acc.: 5

8363 [D loss: 0.657868, acc.: 59.38%] [G loss: 1.011399]
8364 [D loss: 0.676111, acc.: 56.25%] [G loss: 0.953019]
8365 [D loss: 0.676569, acc.: 53.12%] [G loss: 0.899697]
8366 [D loss: 0.675331, acc.: 56.25%] [G loss: 0.891853]
8367 [D loss: 0.748332, acc.: 46.88%] [G loss: 0.831127]
8368 [D loss: 0.677897, acc.: 62.50%] [G loss: 0.818478]
8369 [D loss: 0.723597, acc.: 37.50%] [G loss: 0.678942]
8370 [D loss: 0.752323, acc.: 46.88%] [G loss: 0.864093]
8371 [D loss: 0.708966, acc.: 53.12%] [G loss: 0.881498]
8372 [D loss: 0.578280, acc.: 68.75%] [G loss: 0.908113]
8373 [D loss: 0.557575, acc.: 78.12%] [G loss: 0.869271]
8374 [D loss: 0.710027, acc.: 53.12%] [G loss: 0.881029]
8375 [D loss: 0.866142, acc.: 34.38%] [G loss: 0.845986]
8376 [D loss: 0.648279, acc.: 68.75%] [G loss: 0.956117]
8377 [D loss: 0.687281, acc.: 59.38%] [G loss: 0.877636]
8378 [D loss: 0.641335, acc.: 62.50%] [G loss: 0.878699]
8379 [D loss: 0.717843, acc.: 46.88%] [G loss: 0.875114]
8380 [D loss: 0.615817, acc.: 5

8509 [D loss: 0.636074, acc.: 68.75%] [G loss: 0.977658]
8510 [D loss: 0.697237, acc.: 65.62%] [G loss: 0.890163]
8511 [D loss: 0.676213, acc.: 53.12%] [G loss: 0.956936]
8512 [D loss: 0.615450, acc.: 68.75%] [G loss: 0.880054]
8513 [D loss: 0.691238, acc.: 59.38%] [G loss: 0.849582]
8514 [D loss: 0.748109, acc.: 50.00%] [G loss: 0.851790]
8515 [D loss: 0.631601, acc.: 68.75%] [G loss: 0.903233]
8516 [D loss: 0.652383, acc.: 56.25%] [G loss: 0.878329]
8517 [D loss: 0.660768, acc.: 62.50%] [G loss: 0.925124]
8518 [D loss: 0.663093, acc.: 56.25%] [G loss: 0.930716]
8519 [D loss: 0.722564, acc.: 59.38%] [G loss: 0.845429]
8520 [D loss: 0.689749, acc.: 59.38%] [G loss: 0.872659]
8521 [D loss: 0.671787, acc.: 56.25%] [G loss: 0.782329]
8522 [D loss: 0.657574, acc.: 62.50%] [G loss: 0.759253]
8523 [D loss: 0.661337, acc.: 68.75%] [G loss: 0.844705]
8524 [D loss: 0.675195, acc.: 59.38%] [G loss: 0.889951]
8525 [D loss: 0.609586, acc.: 78.12%] [G loss: 0.853808]
8526 [D loss: 0.674301, acc.: 5

8655 [D loss: 0.666737, acc.: 53.12%] [G loss: 0.876438]
8656 [D loss: 0.699848, acc.: 43.75%] [G loss: 0.790774]
8657 [D loss: 0.646743, acc.: 71.88%] [G loss: 0.862177]
8658 [D loss: 0.726803, acc.: 46.88%] [G loss: 0.905360]
8659 [D loss: 0.682121, acc.: 53.12%] [G loss: 0.974171]
8660 [D loss: 0.725371, acc.: 53.12%] [G loss: 0.886913]
8661 [D loss: 0.656366, acc.: 71.88%] [G loss: 0.852306]
8662 [D loss: 0.612163, acc.: 68.75%] [G loss: 0.906912]
8663 [D loss: 0.581698, acc.: 75.00%] [G loss: 0.917320]
8664 [D loss: 0.618517, acc.: 71.88%] [G loss: 0.885574]
8665 [D loss: 0.767284, acc.: 40.62%] [G loss: 0.879669]
8666 [D loss: 0.709099, acc.: 56.25%] [G loss: 0.830080]
8667 [D loss: 0.608267, acc.: 68.75%] [G loss: 0.893213]
8668 [D loss: 0.752016, acc.: 46.88%] [G loss: 0.810754]
8669 [D loss: 0.718352, acc.: 50.00%] [G loss: 0.825391]


KeyboardInterrupt: 

In [ ]:





# pseudocodice per il paper

# for i in range(batches):
    # z = noise(100)
    # generated_images = Generator(z)
    # (output, teacher_activations) = teacher(generated_images)
    ''' Teacher is the pre-trained network that outputs its activations and the result 
    or it can outputs only the result and we can get the activations with K.function '''
    # combined.train_on_batch(generated_images, (outputs,teacher_activations)) 
    '''Combined is a network that has at the start the generator and then the freezed student.
    The labels it gets are the results and the activations of the teacher
    The loss is to increase the distance between the labels and the output of itself
    therefore this network has to output its intermediate activations'''
    
    # for j in range(ns):
        # student.train_on_batch(generated_images, (outputs,teacher_activations))
        ''' Student network that outputs its results and its intermediate activations, 
        and its loss is to match the output and activations of the teacher'''

    ''' if i don't find a way to output intermediate activations from a model we could always
        get the activations with K.fuction and the data, and then input them to the loss as the label.
        This method does not seems to increase too much the training time...
    '''









